# AI vs Human Text


https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text

*What model of AI has been used to generate the fake text? ==> No infromation*

**RULE: Generated by AI (1) or Human (0)**

## Load dataset

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download shanegerami/ai-vs-human-text

Dataset URL: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text
License(s): other
 97% 338M/350M [00:01<00:00, 222MB/s]
100% 350M/350M [00:01<00:00, 227MB/s]


In [ ]:
! unzip ai-vs-human-text

Archive:  ai-vs-human-text.zip
  inflating: AI_Human.csv            


In [ ]:
import pandas as pd

data=pd.read_csv('AI_Human.csv')
data.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


## Examine Dataset

In [ ]:
data.shape

(487235, 2)

In [ ]:
data["text"].describe()

,text
count,487235
unique,487235
top,cars. cars have been around since they became ...
freq,1


In [ ]:
data["generated"].value_counts()

,count
generated,
0.0,305797
1.0,181438


In [ ]:
(data.duplicated() == True).sum()

0

In [ ]:
data.isnull().sum()

,0
text,0
generated,0


## Transformer

### Hugging face

In [ ]:
from transformers import pipeline
pipe = pipeline(model = "openai-community/roberta-large-openai-detector")

Some weights of the model checkpoint at openai-community/roberta-large-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("openai-community/roberta-large-openai-detector")
tokens = tokenizer.encode(data["text"][0], return_tensors = "tf")
print(tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (717 > 512). Running this sequence through the model will result in indexing errors


tf.Tensor(
[[    0 28986     4  1677    33    57   198   187    51  1059  3395    11
      5 23137    29     6    77 29227  1506    13   417  1412     8  1490
      5    78 11134  6607     4  1677    33   702    10   538   774    11
     84   358   183  1074   187   172     4    53   122     6    82    32
   1158     7   864   114 11361   512  9453    74    28    10   205   631
      4     7   162     6 11361     5   304     9  1677   429    28    10
    205   631     7   109     4 50118 50118   179   101   948     9    42
      6  1566     6    22   179   821  7043 11906     6   301  1411    15
    396  1677    60    30  1615 46370  4533    29 36837   982     6   141
  27596    32     5   784  3796 10188     9 13574     6   147  1692  1380
   1232    31  1169  1481   260 43673    50  1855 42938  3805     7   146
     49  1611     4  2320   224   141    42    16    10  1307 28968  8913
      7   595  1170     7  1888 11832  1123  5035    31  7886 30131     4
   4408  1677    32  2149  

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("openai-community/roberta-large-openai-detector", from_pt=True)

pytorch_model.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
result = model(tokens)
print(result)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.724329 ,  2.2651763]], dtype=float32)>, hidden_states=None, attentions=None)


###Split our data (huge dataset!!)

In [ ]:
data_short =pd.DataFrame(data.head(10000),)
data_short.shape

(10000, 2)

In [ ]:
data_short["generated"].value_counts()

,count
generated,
0.0,8964
1.0,1036


# Try it out on larger dataset

In [ ]:
tokens = tokenizer(data_short["text"].tolist(), max_length = 500, padding = "max_length", truncation = True, return_tensors = "tf")


In [ ]:
tokens["input_ids"].shape

TensorShape([10000, 500])

In [ ]:
embedding = model.predict(tokens["input_ids"])

313/313 [==============================] - 1291s 4s/step


In [ ]:
X = embedding.logits

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, data_short["generated"], test_size = 0.3,stratify=data_short["generated"])

In [ ]:
y_train.value_counts()

,count
generated,
0.0,6275
1.0,725


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)


LogisticRegression()

In [ ]:
result = clf.score(X_test, y_test)
print(round(result,2))

0.9
